# General Preprocessing


In [1]:
%load_ext autoreload
%autoreload 2

## Change to project directory (Google Colab)

In [2]:
import os 
os.chdir('/content/drive/MyDrive/projects/credit-risk-modelling-udemy/notebooks')
os.listdir()

['04-PD-Model-Estimation.ipynb',
 '05-PD-Model-Validation.ipynb',
 '06-PD-Model-Decision-Making.ipynb',
 '01-Exploring-Cleaning-Data.ipynb',
 '03-PD-Model-Data-Preparation.ipynb',
 '07-PD-Model-Monitoring.ipynb',
 '09-LGD-Model.ipynb',
 '08-LGD-EAD-Model-Data-Preparation.ipynb',
 '02-General-Preprocessing.ipynb']

In [5]:
!pip install -e ../.

Obtaining file:///content/drive/MyDrive/projects/credit-risk-modelling-udemy
  Running setup.py develop for src


## Import Libraries

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Load data

In [4]:
from src.project.config import PROJECT_DIR
import os

In [5]:

fpath = os.path.join(PROJECT_DIR, 'data', 'loan_data_2007_2014_cleaned.csv')
df = pd.read_csv(fpath)
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,2011-12-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgrade...,credit_card,Computer,860xx,AZ,27.65,0.0,1985-01-01,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,2015-01-01,171.62,NaN,2016-01-01,0.0,NaN,1,0.0,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,2011-12-01,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use thi...,car,bike,309xx,GA,1.00,0.0,1999-04-01,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,2013-04-01,119.66,NaN,2013-09-01,0.0,NaN,1,0.0,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,2011-12-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,2001-11-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,2014-06-01,649.91,NaN,2016-01-01,0.0,NaN,1,0.0,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,2011-12-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for proper...,other,personel,917xx,CA,20.00,0.0,1996-02-01,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,2015-01-01,357.48,NaN,2015-01-01,0.0,NaN,1,0.0,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,2011-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combini...,other,Personal,972xx,OR,17.94,0.0,1996-01-01,0.0,38.0,NaN,15.0,0.0,27783,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,2016-01-01,67.79,2016-02-01,2016-01-01,0.0,NaN,1,0.0,NaN,NaN,NaN


## Preprocessing few continuous variables

### emp_length

In [6]:
df['emp_length_int'] = df['emp_length'].str.replace(r'years|year', '').str.strip()
df['emp_length_int'] = df['emp_length_int'].str.replace('< 1', '0')
df['emp_length_int'] = df['emp_length_int'].str.replace(r'10\+', '10')
df['emp_length_int'] = df['emp_length_int'].astype(float).astype('Int64')
df[['emp_length', 'emp_length_int']].drop_duplicates().sort_values('emp_length_int')

,emp_length,emp_length_int
1,< 1 year,0
4,1 year,1
25,2 years,2
5,3 years,3
8,4 years,4
10,5 years,5
19,6 years,6
30,7 years,7
6,8 years,8
7,9 years,9


* Works as expected :)
* Missing values will be imputed later

### earliest_cr_line

In [7]:
DATE_OF_ANALYSES = pd.to_datetime('2017-12-01', format='%Y-%m-%d')

In [8]:
def convert_to_time_since(ser, ref_date, units, round_val=True):
    time_diff = (pd.to_datetime(ref_date) - pd.to_datetime(ser))/np.timedelta64(1, units)

    if round_val:
        return time_diff.round().astype('Int64')


In [9]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
df['mths_since_earliest_cr_line'] = DATE_OF_ANALYSES - df['earliest_cr_line']
df['mths_since_earliest_cr_line'].head()

0   12022 days
1    6819 days
2    5874 days
3    7974 days
4    8005 days
Name: mths_since_earliest_cr_line, dtype: timedelta64[ns]

In [10]:
df['mths_since_earliest_cr_line'] = (df['mths_since_earliest_cr_line']/np.timedelta64(1, 'M')).round().astype('Int64')
df['mths_since_earliest_cr_line'].head()

0    395
1    224
2    193
3    262
4    263
Name: mths_since_earliest_cr_line, dtype: Int64

### term

In [11]:
df['term'].unique()

array(['36 months', '60 months'], dtype=object)

In [12]:
df['term_int'] = df['term'].str.replace(' months', '').astype(int).astype('Int64')
df['term_int'].head()

0    36
1    60
2    36
3    36
4    60
Name: term_int, dtype: Int64

### issue_d

In [13]:
df['issue_d'].head()

0    2011-12-01
1    2011-12-01
2    2011-12-01
3    2011-12-01
4    2011-12-01
Name: issue_d, dtype: object

In [14]:
df['mths_since_issue_d'] = convert_to_time_since(df['issue_d'], DATE_OF_ANALYSES, 'M')
df['mths_since_issue_d'].head()

0    72
1    72
2    72
3    72
4    72
Name: mths_since_issue_d, dtype: Int64

## Preprocessing few discrete variables

### addr_state

In [15]:
fpath = os.path.join(PROJECT_DIR, 'data/us_states_info.csv')
us_states_info = pd.read_csv(fpath)
us_states_info.head()

,State Name,USPS Abbreviation,Traditional Abbreviation
0,Alabama,AL,Ala.
1,Alaska,AK,Alaska
2,Arizona,AZ,Ariz.
3,Arkansas,AR,Ark.
4,California,CA,Calif.


In [16]:
us_states_info = pd.merge(us_states_info, df['addr_state'].drop_duplicates(), 
                          left_on='USPS Abbreviation', right_on='addr_state', how='outer', indicator=True)
us_states_info

,State Name,USPS Abbreviation,Traditional Abbreviation,addr_state,_merge
0,Alabama,AL,Ala.,AL,both
1,Alaska,AK,Alaska,AK,both
2,Arizona,AZ,Ariz.,AZ,both
3,Arkansas,AR,Ark.,AR,both
4,California,CA,Calif.,CA,both
5,Colorado,CO,Colo.,CO,both
6,Connecticut,CT,Conn.,CT,both
7,Delaware,DE,Del.,DE,both
8,Florida,FL,Fla.,FL,both
9,Georgia,GA,Ga.,GA,both


In [17]:
us_states_info.loc[us_states_info['_merge'] != 'both']

,State Name,USPS Abbreviation,Traditional Abbreviation,addr_state,_merge
33,North Dakota,ND,N.Dak.,NaN,left_only
50,NaN,NaN,NaN,DC,right_only


This means North Dakota is missing, and we must add this as a category in the addr_state column just for completeness sake

In [18]:
addr_state_cat = pd.Categorical(df['addr_state'], list(us_states_info['USPS Abbreviation'].dropna()) + ['DC'], ordered=False)
addr_state_cat

['AZ', 'GA', 'IL', 'CA', 'OR', ..., 'TX', 'TN', 'OH', 'CA', 'CA']
Length: 466285
Categories (51, object): ['AL', 'AK', 'AZ', 'AR', ..., 'WV', 'WI', 'WY', 'DC']

In [19]:
df.loc[addr_state_cat.isna(), 'addr_state']

Series([], Name: addr_state, dtype: object)

In [20]:
len(addr_state_cat.categories)

51

In [21]:
pd.get_dummies(addr_state_cat).shape

(466285, 51)

* SO, yes! pd.get_dummies will add an extra column for the non-existent state

In [22]:
df['addr_state'] = addr_state_cat

## Get Dummy variables for all categorical variables

In [23]:
cat_columns = ['grade', 'sub_grade', 'home_ownership', 'verification_status', 'loan_status',
               'purpose', 'addr_state', 'initial_list_status']

loan_dummies = []
for col in cat_columns:
    dummy_vars = pd.get_dummies(df[col], prefix=col, prefix_sep=':')
    loan_dummies.append(dummy_vars)

loan_dummies = pd.concat(loan_dummies, axis=1)
loan_dummies.head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G,sub_grade:A1,sub_grade:A2,sub_grade:A3,sub_grade:A4,sub_grade:A5,sub_grade:B1,sub_grade:B2,sub_grade:B3,sub_grade:B4,sub_grade:B5,sub_grade:C1,sub_grade:C2,sub_grade:C3,sub_grade:C4,sub_grade:C5,sub_grade:D1,sub_grade:D2,sub_grade:D3,sub_grade:D4,sub_grade:D5,sub_grade:E1,sub_grade:E2,sub_grade:E3,sub_grade:E4,sub_grade:E5,sub_grade:F1,sub_grade:F2,sub_grade:F3,sub_grade:F4,sub_grade:F5,sub_grade:G1,sub_grade:G2,sub_grade:G3,...,addr_state:IN,addr_state:IA,addr_state:KS,addr_state:KY,addr_state:LA,addr_state:ME,addr_state:MD,addr_state:MA,addr_state:MI,addr_state:MN,addr_state:MS,addr_state:MO,addr_state:MT,addr_state:NE,addr_state:NV,addr_state:NH,addr_state:NJ,addr_state:NM,addr_state:NY,addr_state:NC,addr_state:ND,addr_state:OH,addr_state:OK,addr_state:OR,addr_state:PA,addr_state:RI,addr_state:SC,addr_state:SD,addr_state:TN,addr_state:TX,addr_state:UT,addr_state:VT,addr_state:VA,addr_state:WA,addr_state:WV,addr_state:WI,addr_state:WY,addr_state:DC,initial_list_status:f,initial_list_status:w
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


### Concat with the original data

In [24]:
df.shape

(466285, 61)

In [25]:
df = pd.concat([df, loan_dummies], axis=1)
df.shape

(466285, 188)

## Fill Missing Values

In [26]:
miss_df = df.isna().sum()
miss_df[miss_df != 0]

emp_title                       27588
emp_length                      21008
annual_inc                          4
desc                           340538
title                              20
delinq_2yrs                        29
earliest_cr_line                   29
inq_last_6mths                     29
mths_since_last_delinq         250351
mths_since_last_record         403647
open_acc                           29
pub_rec                            29
revol_util                        340
total_acc                          29
last_pymnt_d                      376
next_pymnt_d                   227214
last_credit_pull_d                 42
collections_12_mths_ex_med        145
mths_since_last_major_derog    367311
acc_now_delinq                     29
tot_coll_amt                    70276
tot_cur_bal                     70276
total_rev_hi_lim                70276
emp_length_int                  21008
mths_since_earliest_cr_line        29
dtype: int64

### total_rev_hi_lim

* Not sure why funded amount is used to fill in the NaNs here...

In [27]:
df['total_rev_hi_lim'].fillna(df['funded_amnt'], inplace=True)

In [28]:
df['annual_inc'].fillna(df['annual_inc'].mean(), inplace=True)

In credit risk, we follow a conservative approach to risk. So we will NaNs with values which would make the loan less suitable (and hence decrease credit score)

In [29]:
df['mths_since_earliest_cr_line'].fillna(0, inplace=True)
df['acc_now_delinq'].fillna(0, inplace=True)
df['total_acc'].fillna(0, inplace=True)
df['pub_rec'].fillna(0, inplace=True)
df['open_acc'].fillna(0, inplace=True)
df['inq_last_6mths'].fillna(0, inplace=True)
df['delinq_2yrs'].fillna(0, inplace=True)
df['emp_length_int'].fillna(0, inplace=True)

In [30]:
miss_df = df.isna().sum()
miss_df[miss_df != 0]

emp_title                       27588
emp_length                      21008
desc                           340538
title                              20
earliest_cr_line                   29
mths_since_last_delinq         250351
mths_since_last_record         403647
revol_util                        340
last_pymnt_d                      376
next_pymnt_d                   227214
last_credit_pull_d                 42
collections_12_mths_ex_med        145
mths_since_last_major_derog    367311
tot_coll_amt                    70276
tot_cur_bal                     70276
dtype: int64

We will ignore the above columns

## loan_status

In [31]:
df['loan_status'].value_counts(normalize=True)

Current                                                0.480878
Fully Paid                                             0.396193
Charged Off                                            0.091092
Late (31-120 days)                                     0.014798
In Grace Period                                        0.006747
Does not meet the credit policy. Status:Fully Paid     0.004263
Late (16-30 days)                                      0.002612
Default                                                0.001784
Does not meet the credit policy. Status:Charged Off    0.001632
Name: loan_status, dtype: float64

* `Current`, `Fully Paid`, `In Grace Period`, `Does not meet the credit policy. Status:Fully Paid`
    * `Good` Loans - Non-Default
* `Charged Off`, `Late (31-120 days)`, `Late (16-30 days)`, `Default`, `Does not meet the credit policy. Status:Charged Off`
    * `Bad` Loans - Default


In [32]:
good_loan_cats = ['Current', 'Fully Paid', 'In Grace Period', 'Does not meet the credit policy. Status:Fully Paid']
df['good_bad_loan'] = df['loan_status'].isin(good_loan_cats).astype('Int64')
df['good_bad_loan'].value_counts(normalize=True)

1    0.888081
0    0.111919
Name: good_bad_loan, dtype: float64

* The Dataset is highly imbalanced with only 11% defaults
* Assigning 1 to a good loan is done purposely to ensure that the weights associated with each feature when higher (positive), should imply a positive association with being a good loan
    * and hence in turn imply a higher credit score

In [33]:
out_file = os.path.join(PROJECT_DIR, 'outputs', 'loan_data_general_preprocessed_nb2.pkl')
df.to_pickle(out_file)